In [ ]:
import pandas as pd
import pyterrier as pt
import os
import requests
import json
import re
import csv
import numpy as np

In [ ]:
if not pt.started():
  pt.init()

In [ ]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './indices/cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
  indexer = pt.index.IterDictIndexer(pt_index_path, blocks = True)
  index_ref = indexer.index(dataset.get_corpus_iter(),
                            fields = ['title', 'doi', 'abstract'],
                            meta = ('docno',))
else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

In [ ]:
metadata = pd.read_csv('~/.ir_datasets/cord19/2020-07-16/metadata.csv')

In [ ]:
len(metadata)

In [ ]:
metadata.head(5)

Dropping empty autors

In [ ]:
metadata = metadata[metadata['authors'].notna()]
metadata = metadata.reset_index()
metadata = metadata.drop(columns = 'index')

In [ ]:
journal_abbrev = pd.read_csv('./data/wos_abbrev_table.csv', sep = ';', header = None)
journal_abbrev=journal_abbrev.T.set_index(0).T
journal_abbrev = journal_abbrev[['full', 'abbrev']]

In [ ]:
metadata = pd.merge(metadata, journal_abbrev, left_on='journal', right_on='abbrev', how='left')
metadata['full'] = metadata['full'].fillna(metadata['journal'])


In [ ]:
metadata

## Splitting the authors into single columns

A function to split the author dict up into single columns. To make the retrieval process possible, commas, double spaces and other characters had to be removed without using a complex regex solution (in case a person hat a accent in their name for example.

In [ ]:
def split_authors(n):
    # get first n articles
    author_uid_df = metadata[['authors', 'cord_uid']][:n]
    author_uid_df['authors'] = author_uid_df['authors'].apply(pd.Series)
    # stringsplit authors
    splitted_authors_df = author_uid_df['authors'].str.split(';', expand=True)
    author_uid_df = pd.concat([author_uid_df, splitted_authors_df], axis=1)
    # drop the authors (column with multiple authors)
    author_uid_df.drop(columns='authors')
    # replace the empty column of multiple authors
    author_uid_df = author_uid_df.drop(columns='authors')
    author_uid_df = author_uid_df.replace('', np.nan).set_index('cord_uid').stack().reset_index(name='author').drop('level_1',1)
    author_uid_df['author'] = author_uid_df['author'].str.split(',').str[::-1].str.join(' ')

    return author_uid_df

In [ ]:
def split_authors(n):
    # get first n articles
    author_uid_df = metadata[['authors', 'cord_uid']][:n]
    author_uid_df['authors'] = author_uid_df['authors'].apply(pd.Series)
    # stringsplit authors
    splitted_authors_df = author_uid_df['authors'].str.split(';', expand=True)
    # concatenate the splitted authors to our dataframe
    author_uid_df = pd.concat([author_uid_df, splitted_authors_df], axis=1)
    # drop the authors (column with multiple authors)
    author_uid_df.drop(columns='authors')
    # replace the empty column of multiply authors
    author_uid_df = author_uid_df.drop(columns='authors')
    author_uid_df = author_uid_df.replace('', np.nan).set_index('cord_uid').stack().reset_index(name='author').drop('level_1',1)
    author_uid_df['author'] = author_uid_df['author'].str.split(',').str[::-1].str.join(' ')
    author_uid_df['author'] = author_uid_df['author'].str.replace('  ', ' ')
    author_uid_df['author'] = author_uid_df['author'].str.lstrip()
    merged_data = metadata[['cord_uid', 'title', 'license', 'publish_time', 'full']]
    merged_data = author_uid_df.merge(merged_data, on = 'cord_uid')
    return merged_data

### How to work with the API:

Basic request:
https://api.openalex.org/ + work/authors/venues/insitutions/concecpts + ?filter = + columns.search + Name

In [ ]:
tariq = requests.get(
    'https://api.openalex.org/authors?filter=display_name.search:Tariq'
).json()['results'][0]                     

## Retrieving author information

A function to retrieve all the authors from the metadata's first n documents. It creates a list to call OpenAlex with.

In [ ]:
def getAuthors(n_documents):
    author_list = []
    for i in range(n_documents):
        print(metadata['authors'][i])
        if(metadata['authors'][i] != "nan"):
            authors_split = metadata['authors'][i].split(";")
            for j in range(len(authors_split)):
                authors_split[j] = authors_split[j].replace(',', '')
                authors_split[j] = authors_split[j].lstrip()
                author_list.append(authors_split[j])
        else:
            author_list.append("")
    return author_list

In [ ]:
# Gets all splitted authors from the first 500 documents
authors = getAuthors(500)

Retrieving the data using OpenAlex. If the author is not found, Empty Name is used as dummy.

In [ ]:
author_df = pd.DataFrame()

for i in range(len(authors)):
    try:
        author_information = requests.get(
            'https://api.openalex.org/authors?filter=display_name.search:'+authors[i]
        ).json()['results'][0]
        print('Retrieved:',authors[i])
        current_author = pd.DataFrame.from_dict(author_information, orient='index')
        current_author = current_author.transpose()
        author_df = author_df.append(current_author)
        author_df['display_name'] =  author_df['display_name'].str.lstrip()

        
    except:
        print("Empty name")


In [ ]:
author_df

Cleaning up the author data: 

1. Defining a dataframe
2. add the extracted information into our dataframe
3. replace special characters in the display name
4. add the x_concepts (most common research field of the author according to openalex)

In [ ]:
author_data = pd.DataFrame(columns= [['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution', 'x_concepts']])
author_data = author_df[['display_name', 'relevance_score', 'works_count', 'cited_by_count', 'last_known_institution', 'x_concepts']]

author_data['research_field'] = ""
author_data = author_data.reset_index()
author_data = author_data.drop(columns = 'index')


#clean display names again

for i in range(len(author_data)):
    author_data['display_name'][i] = author_data['display_name'][i].lstrip()

for i in range(len(author_data)):
    try:
        author_data.iloc[i, author_data.columns.get_loc('research_field')] = author_data.iloc[i]['x_concepts'][0]['display_name']
    except:
        author_data.iloc[i, author_data.columns.get_loc('research_field')] = "Unknown"
    try:
        author_data.iloc[i, author_data.columns.get_loc('last_known_institution')] = (list(author_data['last_known_institution'])[i]['display_name'])    
    except:
        author_data.iloc[i, author_data.columns.get_loc('last_known_institution')] = "Unknown"
author_data = author_data.drop(columns = ['x_concepts']) 

In [ ]:
author_data

## Retrieving Insitution Information

Create the institution list from the author data

In [ ]:
institutions_list = author_data['last_known_institution'].unique()

Replace commas and spaces at the start to retrieve the documents.

In [ ]:
for i in range(len(institutions_list)):
    institutions_list[i] = institutions_list[i].replace(',','')
    institutions_list[i] = institutions_list[i].lstrip()
    print(institutions_list[i])

Create a dataframe and retrieve the institutions using OpenAlex.

In [ ]:
institution_df = pd.DataFrame()

for i in range(len(institutions_list)):
    try:
        print('Retrieved:',institutions_list[i])
        institution_information = requests.get(
            'https://api.openalex.org/institutions?filter=display_name.search:'+institutions_list[i]
        ).json()['results'][0]
        current_institution = pd.DataFrame.from_dict(institution_information, orient='index')
        current_institution = current_institution.transpose()
        institution_df = institution_df.append(current_institution)
    except:
        print("Empty name")


Preprocess the institution data by:

1. renaming
2. find out the main research field
3. replacing special characters

In [ ]:
institution_data = institution_df[['display_name', 'relevance_score', 'country_code', 'type', 'cited_by_count', 'x_concepts']]
institution_data = institution_data.rename(columns= {'display_name' : 'institution_name', 'relevance_score': 'institution_relevance_score', 'cited_by_count': 'institution_cited_by_count', 'x_concepts' : 'institution_main_research_field'})

# gain knowledge about the institutions main research field
for i in range(len(institution_data)):
    try:
        institution_data.iloc[i, institution_data.columns.get_loc('institution_main_research_field')] = (list(institution_data['institution_main_research_field'])[i][0]['display_name'])    
    except:
        institution_data.iloc[i, institution_data.columns.get_loc('institution_main_research_field')] = "Unknown"

institution_data = institution_data.reset_index()
institution_data = institution_data.drop(columns = 'index')
        
institution_data.institution_name = institution_data.institution_name.str.replace('[^a-zA-Z]', ' ')
institution_data.institution_name = institution_data.institution_name.str.replace('  ',' ')

In [ ]:
institution_data

## Combining the dataframes:

Combining the data was a time consuming task due to smaller errors and mistakes.
1. taking the authors from the metadata and replace their special characters
2. taking the authors from the retrieved authors to gain more knowledge about them
3. compare lost data due to merges (e.g. missing values somewhere or wrong name alignments)
    - 3.1 since the metadata is taken document wise, high amounts of authors could be lost. Then, more have to be retrived
4. merging the data 
5. comparing lost data due to merges in terms of adding the institution
6. merging the data again now containing information about the Article, Authors and Institutions

In [ ]:
#replace metadata authors 
metadata_authors = split_authors(500)#len(metadata))
metadata_authors.author = metadata_authors.author.str.replace('  ',' ')
metadata_authors['author'] = metadata_authors['author'].str.lstrip()
metadata_authors.author = metadata_authors.author.str.replace('.', '')

metadata_authors = metadata_authors.rename(columns={'author': 'display_name'})

In [ ]:
# replacing double spaces and dots
author_data.display_name = author_data.display_name.str.replace('.', '')
author_data.last_known_institution = author_data.last_known_institution.str.replace('  ',' ')

lost data due to name merges:

In [ ]:
metadata_authors

In [ ]:
metadata_authors['display_name']

In [ ]:
author_data['display_name']

Lost data due to authors: eventually high if split_authors has a higher value than getAuthors.

In [ ]:
len(list(set(metadata_authors['display_name']) - set(author_data['display_name'])))

In [ ]:
#merging our dataframes

combined_data = metadata_authors.merge(author_data, on = 'display_name', how = 'outer')

lost data due to institution:

In [ ]:
list(set(combined_data['last_known_institution']) - set(institution_data['institution_name']))

In [ ]:
combined_data2 = combined_data.merge(institution_data, left_on = 'last_known_institution', right_on = 'institution_name', how = 'outer')

In [ ]:
combined_data2

## Retrieving Journal information

In [ ]:
journal_list = combined_data['full'].unique()

In [ ]:
journal_df = pd.DataFrame()

for i in range(len(journal_list)):
    try:
        print(journal_list[i])
        journal_information = requests.get(
            'https://api.openalex.org/venues?filter=display_name.search:'+journal_list[i]
        ).json()['results'][0]
        current_journal = pd.DataFrame.from_dict(journal_information, orient='index')
        current_journal = current_journal.transpose()
        journal_df = journal_df.append(current_journal)
    except:
        print("Empty name")


In [ ]:
journal_data = journal_df[['issn', 'display_name', 'publisher', 'relevance_score', 'cited_by_count', 'cited_by_count','x_concepts']]
journal_data = journal_data.rename(columns = {'display_name':'journal_display_name', 'publisher': 'journal_publisher', 'relevance_score': 'journal_relevance_score', 'cited_by_count' : 'journal_cited_count', 'x_concepts' : 'main_research_field'})

In [ ]:
journal_data

In [ ]:
for i in range(len(journal_data)):
    try:
        journal_data.iloc[i, journal_data.columns.get_loc('main_research_field')] = (list(journal_data['main_research_field'])[i][0]['display_name'])    
    except:
        journal_data.iloc[i, journal_data.columns.get_loc('main_research_field')] = "Unknown"

Merging the journal data now to our complete dataframe

In [ ]:
# replacing dots
journal_data.journal_display_name = journal_data.journal_display_name.str.replace('.', '')

In [ ]:
list(set(combined_data2['full']) - set(journal_data['journal_display_name']))

In [ ]:
combined_data3 = combined_data2.merge(journal_data, left_on = 'full', right_on = 'journal_display_name', how = 'outer')
combined_data3 = combined_data3.loc[:,~combined_data3.columns.duplicated()]

### Our completed Dataframe consisting of: 



In [ ]:
combined_data3.info()

In [ ]:
#combined_data3.to_csv('outer_full_dataframe_500_new.csv', index=False, header=True)
